In [165]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import numpy as np

In [166]:
MAX_LOOKBACK = 50
##select = RFE(RandomForestRegressor(n_estimators=100, random_state=42),
                ## n_features_to_select=10)

In [ ]:
    def read_stock_data():
        '''
        Description:
            Reads in simulated stock data from stock_data.csv
        Returns:
            stock_df (DataFrame): standardized ticker/factor data in pandas df
        Raises:
            AssertionError: ticker_data.csv/factor_data.csv has an invalid format
        '''
        ticker_df = pd.read_csv('stock_data/ticker_data.csv')
        factor_df = pd.read_csv('stock_data/factor_data.csv')
        assert 'timestep' in ticker_df.columns, "ticker_data.csv has an invalid format"
        assert 'ticker' in ticker_df.columns, "ticker_data.csv has an invalid format"
        assert 'returns' in ticker_df.columns, "ticker_data.csv has an invalid format"
        assert 'timestep' in factor_df.columns, "factor_data.csv has an invalid format"
        ticker_df.set_index('timestep', inplace=True)
        factor_df.set_index('timestep', inplace=True)
        stock_df = ticker_df.join(factor_df, how='left')
        return stock_df

In [168]:
stock_df=read_stock_data();

In [169]:
    def build_signal(stock_features):
        enc = LabelBinarizer()
        enc_results = enc.fit_transform(stock_features['industry'])
        
        industry_binary = pd.DataFrame(enc_results, columns=enc.classes_)
        stock_features.drop('industry', axis=1, inplace=True)
        feature_binary = stock_features.join(industry_binary, how='left')
        feature_binary.fillna(0, inplace=True)
        
        returns = feature_binary.set_index('ticker')['returns']
        
        X_train, X_test, y_train, y_test = train_test_split(feature_binary, returns, random_state = 0)
        
        rf = RandomForestRegressor().fit(X_train, y_train)
        
        print("Score on training set for Unscaled RandomForestRegressor:", rf.score(X_train, y_train))
        print("Score on test set for Unscaled RandomForestRegressor:", rf.score(X_test, y_test))
        print("Importance of features:", rf.feature_importances_)
        #feature_importance = pd.DataFrame(rf.feature_importances_,
                                          index = X_train.columns,
                                          columns=['importance']).sort_values('importance', ascending=False)
        #return feature_importance.head()

In [171]:
    def simulate_portfolio():
        '''
        Description:
            Simulates performance of the portfolio on historical data
        Return:
            sharpe (int) - sharpe ratio for the portfolio
        '''
        daily_returns = []
        stock_df = read_stock_data()
        for idx in stock_df.index.unique():
            #print("timestep", idx)
            if idx < MAX_LOOKBACK:
                continue
            stock_features = stock_df.loc[idx-MAX_LOOKBACK:idx-1]
            returns = stock_df.loc[idx:idx].set_index('ticker')['returns']
            
            signal = build_signal(stock_features)
            #signal_return = returns * signal
            #daily_returns.append(np.mean(signal_return))
        #sharpe_ratio = np.sqrt(252) * (np.mean(daily_returns) / np.std(daily_returns))
        return signal

In [172]:
simulate_portfolio()

/Users/qiaqia/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/qiaqia/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/qiaqia/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/qiaqia/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set o

KeyboardInterrupt: 